In [7]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Sep 12 22:46:37 2019

@author: andreamourelo
"""

from IPython.display import display, Markdown, Latex
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings('ignore')
import csv
import numpy as np
import sys, getopt
import math
import random
import timeit
import os


instance = ''
nNodes = 0
nVehicles = 0
nGroups = 0
capacity = 0
nodes = []
groups = []
globalDemands = []
distances = []

######## FUNCTIONS
# Função usada para o set up da seed a cada função, pois se não não funcionava
def set_seed(seed_value):
    # Set a seed value: 
    seed_value = seed_value  
    # 1. Set PYTHONHASHSEED environment variable at a fixed value: 
    os.environ['PYTHONHASHSEED']=str(seed_value) 
    # 2. Set python built-in pseudo-random generator at a fixed value:
    random.seed(seed_value) 
    # 3. Set numpy pseudo-random generator at a fixed value:
    np.random.seed(seed_value)    
        
def calculateAngle(x,y):
    angle = 0
    pi = math.pi
    arctanXY = math.atan(y/x)
    if x == 0:
        if y > 0:
            angle = pi/2
        else :
            angle = 3*pi/2
    elif x > 0:
        if y >= 0:
            angle = arctanXY
        if y < 0:
            angle = arctanXY + 2*pi
    else:
        angle = arctanXY + pi
    return angle
        
        
def createInitialSolutionBySweep(nVehicles, nGroups, capacity, demands, nodes):
    angles = {}
    # Create dictionary with "nodeNumber : (angle, group)"
    for i in range(1,len(nodes)): # Not inserting the depot (0)
        node = nodes[i]
        angle = calculateAngle(node[0], node[1])
        angles.update({i : (angle, node[2])}) 
    sortedAngles = sorted(angles.items(), key=lambda x:x[1])
    # Keeping track of groups already assigned
    groupsAssigned = {}
    for group in range(1,nGroups + 1): # the depot is group 0
        groupsAssigned.update({group : False})
    # Inserting routes
    routes = []
    demandDistribution = []
    demandDistributionOfGroups = []
    nextRoute = []
    demandOfRoute = []
    totalDemandOfRoute = 0
    for nodeData in sortedAngles: # gets (node,(angle,group))
        nodeGroup = nodeData[1][1]
        if groupsAssigned[nodeGroup] == False: # not yet assigned
            groupDemand = demands[nodeGroup - 1] 
            if groupDemand + totalDemandOfRoute < capacity:
                nextRoute.append(nodeGroup)
                demandOfRoute.append(groupDemand)
                totalDemandOfRoute += groupDemand
            else:
                routes.append(nextRoute)
                demandDistributionOfGroups.append(demandOfRoute)
                demandDistribution.append(totalDemandOfRoute)
                nextRoute = [nodeGroup]
                demandOfRoute = [groupDemand]
                totalDemandOfRoute = groupDemand
            groupsAssigned[nodeGroup] = True
    routes.append(nextRoute)
    demandDistributionOfGroups.append(demandOfRoute)
    demandDistribution.append(totalDemandOfRoute)
    
    if len(routes) > nVehicles:
        capacityMissing = demandDistribution[-1] - (capacity - demandDistribution[-2])
        #print(capacityMissing, routes)
        #print(" demandDistribution: ",demandDistribution)
        #print("demandDistributionOfGroups : ",demandDistributionOfGroups)
        route1 = routes[-2]
        demandProblem = demandDistribution[-1]
        for i in range(len(route1)):
            group1 = route1[i]
            demand1 = demandDistributionOfGroups[-2][i]
            for j in range(len(routes[:-2])):
                route2 = routes[j]
                for k in range(len(route2)):
                    group2 = route2[k]
                    demand2 = demandDistributionOfGroups[j][k]
                    diff = demand1 - demand2
                    newDemandGroup2 = demandDistribution[j] - demand2 + demand1
                    if diff > 0 and diff >= capacityMissing and newDemandGroup2 < capacity:
                        print(route1, route2, group1, group2, diff)
                        route1[i] = group2
                        route2[k] = group1
                        demandDistributionOfGroups[j][k] = demand1
                        demandDistributionOfGroups[-2][i] = demand2
                        demandDistribution[-2] = demandDistribution[-2] + demand2 - demand1
                        demandDistribution[j] = demandDistribution[j] + demand1 - demand2
                        
                        for group in routes[-1]:
                            routes[-2].append(group)
                        routes = routes[:-1]
                        demandDistribution[-2] += demandProblem
                        #demandDistributionOfGroups = demandDistributionOfGroups[:-1]
                        demandDistribution = demandDistribution[:-1]
                        
                        #print("routes : ", routes)
                            
                        #print(" demandDistribution: ",demandDistribution)
                        #print("demandDistributionOfGroups : ",demandDistributionOfGroups)
                        return routes, demandDistribution
    return routes, demandDistribution
        
        

def localSearch(routes,distancesPerVehicle):
    routes_final = routes
    min_distances = distancesPerVehicle
    end = False
    iterations = 0
    while (end != True):
        routes_final, min_distances, changedTwoOpt = twoOptLocalSearch(routes_final,min_distances)
        routes_final, min_distances, changedCross = crossLocalSearch(routes_final, min_distances)
        if changedCross == False:
            routes_final, min_distances, changedTwoOptStar = twoOptStarLocalSearch(routes_final, min_distances)
            if changedTwoOptStar == False:
                end = True
        iterations +=1
    print("local search made x iterations with x = ", iterations)
    return routes_final, min_distances

def twoOptStarLocalSearch(routes_final, min_distances):
    changed = False
    route_i,route_j = -1,-1  # servem para contar onde estamos ao percorrer as rotas
    group_i, group_j = -1,-1 # servem para contar onde estamos ao percorrer os grupos de cada rota
    for route1 in routes_final:
        route_i += 1
        for route2 in routes_final:
            route_j += 1
            if route1 != route2:
                for group1 in route1:
                    group_i += 1
                    for group2 in route2:
                        group_j += 1
                        #if group1 != 0 and group2 != 0:
                        new_route1, new_route2 = twoOptStar(route1,route2,group_j)
                        # We need to check if the new_routes are feasible
                        #print(i,j)
                        if isFeasible(new_route1) and isFeasible(new_route2):
                            new_distance1, new_distance2 = calculateDistanceAfterTwoOptStar(new_route1, new_route2, group_i, group_j)
                            if new_distance1 + new_distance2 < min_distances[route_i] + min_distances[route_j]:
                                #print("new route thanks to 2optStar : ",new_route1, new_route2)
                                routes_final[route_i] = new_route1
                                routes_final[route_j] = new_route2
                                min_distances[route_i] = new_distance1
                                min_distances[route_j] = new_distance2
                                changed = True
                                return routes_final,min_distances,changed
                    group_j = -1
        group_i, group_j = -1,-1
        route_j = -1
    return routes_final,min_distances,changed


def crossLocalSearch(routes_final, min_distances):
    changed = False
    route_i,route_j = -1,-1  # servem para contar onde estamos ao percorrer as rotas
    group_i, group_j = -1,-1 # servem para contar onde estamos ao percorrer os grupos de cada rota
    for route1 in routes_final:
        route_i += 1
        for route2 in routes_final:
            route_j += 1
            if route1 != route2:
                for group1 in route1:
                    group_i += 1
                    for group2 in route2:
                        group_j += 1
                        #if group1 != 0 and group2 != 0:
                        new_route1, new_route2 = cross(route1,route2,group_i,group_j)
                        # We need to check if the new_routes are feasible
                        if isFeasible(new_route1) and isFeasible(new_route2):
                            new_distance1, new_distance2 = calculateDistanceAfterCross(new_route1, new_route2, group_i, group_j)
                            if new_distance1 + new_distance2 < min_distances[route_i] + min_distances[route_j]:
                                #print("new route thanks to cross : ",route1, route2, new_route1, new_route2, group_i, group_j)
                                routes_final[route_i] = new_route1
                                routes_final[route_j] = new_route2
                                min_distances[route_i] = new_distance1
                                min_distances[route_j] = new_distance2
                                changed = True
                                return routes_final,min_distances,changed
                    group_j = -1
        group_i, group_j = -1,-1
        route_j = -1
    return routes_final,min_distances,changed

def twoOptLocalSearch2(routes_final,min_distances):
    print("hola")
    # 2-opt em todas as rotas
    changed = False
    for num_route in range(len(routes_final)):
        route_iteration = routes_final[num_route]
        min_distance = min_distances[num_route]
        group_i = -1
        for group1 in route_iteration:
            group_i +=1
            group_j = group_i + 1
            for group2 in route_iteration[group_j:]: # tentando twoOpt com todos os outros da rota
                #group1, group2 = route_iteration[group_i], route_iteration[group_j]
                #print(group_i, group_j)
                new_route = twoOptIntra(route_iteration, group_i, group_j)
                new_distance = calculateDistanceAfter2Opt(new_route, group_i, group_j)
                if new_distance < min_distance:
                    #print("nova rota thanks to 2opt : ",new_route, new_distance)
                    route_iteration = new_route
                    min_distance = new_distance
                    routes_final[num_route] = route_iteration
                    min_distances[num_route] = min_distance
                    changed = True
                group_j += 1
    return routes_final, min_distances, changed

def twoOptLocalSearch(routes_final,min_distances):
    # 2-opt em todas as rotas
    changed = False
    for num_route in range(len(routes_final)):
        route_iteration = routes_final[num_route]
        min_distance = min_distances[num_route]
        group_i, group_j = -1, -1
        for group1 in route_iteration:
            group_i +=1
            for group2 in route_iteration: # tentando twoOpt com todos os outros da rota
                group_j +=1
                if group_i != group_j :
                    new_route = twoOptIntra(route_iteration, group_i, group_j)
                    new_distance = calculateDistanceAfter2Opt(new_route, group_i, group_j)
                    if new_distance < min_distance:
                        route_iteration = new_route
                        min_distance = new_distance
                        routes_final[num_route] = route_iteration
                        min_distances[num_route] = min_distance
                        changed = True
            group_j = -1
    return routes_final, min_distances, changed

def calculateDistanceAfter2Opt(new_route, group1, group2):
    new_distance = calculateDistancesForRoute(new_route, distances)
    return new_distance
  
def calculateDistanceAfterTwoOptStar(new_route1, new_route2, group_i, group_j):
    new_distance1, new_distance2 = calculateDistancesForRoute(new_route1, distances), calculateDistancesForRoute(new_route2, distances)
    return new_distance1, new_distance2

def calculateDistanceAfterCross(new_route1, new_route2, group1, group2):
    new_distance1, new_distance2 = calculateDistancesForRoute(new_route1, distances), calculateDistancesForRoute(new_route2, distances)
    return new_distance1, new_distance2

def calculateDistanceAfterExchange(new_route1, new_route2):
    new_distance1, new_distance2 = calculateDistancesForRoute(new_route1, distances), calculateDistancesForRoute(new_route2, distances)
    return new_distance1, new_distance2

def calculateDistancesForRoute(route, distances):
    fullRoute = [0] + route + [0]
    p = FloydWarshall(distances.copy(), fullRoute.copy())
    totalDistanceOfVehicle = p[0][0]
    return totalDistanceOfVehicle

def calculateDistancesForRoutes(routes, distances):
    distancesPerVehicle = np.zeros(len(routes))
    for vehicle in range(len(routes)):
        fullRoute = [0] + routes[vehicle] + [0]
        p = FloydWarshall(distances.copy(), fullRoute.copy())
        totalDistanceOfVehicle = p[0][0]
        distancesPerVehicle[vehicle] = totalDistanceOfVehicle
    return distancesPerVehicle

def perturbate2(routes, numberOfPerturbationsDone,distancesPerVehicle, timePassed, timeMax):
    # Bota um elemento de uma rota em todas as outras rotas e testa
    routes_final = routes
    min_distances = distancesPerVehicle
    feasibleSolution = False
    #for i in range(10):
    time = timePassed
    while feasibleSolution == False and time < timeMax:
        inicio = timeit.default_timer()
        numberOfRoutes = len(routes)
        routesIdentifiers = [i for i in range(numberOfRoutes)]
        for route_i in range(len(routes_final)):
            route1 = routes_final[route_i]
            for group1 in route1:
                for route_j in range(len(routes_final)):
                    route2 = routes_final[route_j]
                    copy_route1 = route1.copy()
                    if route1 != route2:
                        copy_route1.remove(group1)
                        new_route1, new_route2 = copy_route1, route2 + [group1]
                        numberOfPerturbationsDone += 1 
                        if isFeasible(new_route1) and isFeasible(new_route2):
                            new_distance1, new_distance2 = calculateDistanceAfterExchange(new_route1, new_route2)
                            if new_distance1 + new_distance2 < min_distances[route_i] + min_distances[route_j]:
                                routes_final[route_i] = new_route1
                                routes_final[route_j] = new_route2
                                min_distances[route_i] = new_distance1
                                min_distances[route_j] = new_distance2
                                feasibleSolution = True
                                print("found a solution when perturbating : ", new_route1, new_route2)
                                return routes_final, min_distances, numberOfPerturbationsDone, time
                                
                                #return routes_final, min_distances, numberOfPerturbationsDone
        fim = timeit.default_timer()
        time += fim - inicio
    print("No solution found with perturbation")
    return routes_final, min_distances, numberOfPerturbationsDone, time


def twoOptIntra(route, group1, group2):
    # Will remove and replace 2 edges in the route
    # example: with route = [1,2,3,4,5,6,7,8], twoOptIntra(route, 2, 5) returns [1 2 3 6 5 4 7 8]
    #print(route, group1,group2)
    if (group2 < group1):
        temp = group1
        group1 = group2
        group2 = temp
    #print(group1,group2)
    temporary = route[group1 + 1 : group2]
    #print("temporary : ",temporary)
    if temporary != []:
        new_route = route[:group1 + 1] + [route[group2]] + temporary[::-1] + route[group2 + 1 : ]
    else:
        return route
    return new_route

def twoOptStar(route1, route2, group):
    # Will remove and replace 2 consecutive edges in two different routes
    # example : with route1 = [1,2,3,4,5,6,7,8] and route2 = [9,10,11,12]
    # twoOptInter(route1, route2, 1)
    # will give (array([ 1,  2, 11, 12]), array([ 9, 10,  3,  4,  5,  6,  7,  8]))
    new_route1 = route1[:group + 1] + route2[group + 1 :]
    new_route2 = route2[:group + 1] + route1[group + 1 :]
    return new_route1, new_route2

def twoOptStarForCross(route1, route2, group):
    # Will remove and replace 2 consecutive edges in two different routes
    # example : with route1 = [1,2,3,4,5,6,7,8] and route2 = [9,10,11,12]
    # twoOptInter(route1, route2, 1)
    # will give (array([ 1,  2, 11, 12]), array([ 9, 10,  3,  4,  5,  6,  7,  8]))
    group +=1
    new_route1 = route1[:group + 1] + route2[group + 1 :]
    new_route2 = route2[:group + 1] + route1[group + 1 :]
    return new_route1, new_route2

def cross(route1, route2, group1, group2):
    # Will exchange 2 sequences of visits
    # example : with route1 = [1,2,3,4,5,6,7,8] and route2 = [9,10,11,12]
    # cross(route1, route2, 1, 3)
    # will return ([1, 2, 11, 12, 5, 6, 7, 8], [9, 10, 3, 4])    
    if (group2 < group1):
        temp = group2
        group2 = group1
        group1 = temp
    route1_temporary, route2_temporary = twoOptStar(route1, route2, group1)
    route1_final, route2_final = twoOptStar(route1_temporary,route2_temporary, group2)
    return route1_final, route2_final

def exchange(route1, route2, group1, group2):
    # Will exchange 2 groups of 2 routes
    # example : with route1 = [1,2,3,4,5,6,7,8] and route2 = [9,10,11,12]
    # exchange(route1, route2, 2, 3)
    # will return ([1,2,12,4,5,6,7,8], [9,10,11,3])    
    tempGroupRoute1 = route1[group1]
    tempGroupRoute2 = route2[group2]
    route1[group1] = tempGroupRoute2
    route2[group2] = tempGroupRoute1
    return route1, route2

def isFeasible(route):
    '''for group in route:
        if group == 0:
            return False'''
    demandOfRoute = 0
    for group in route:
        demandOfRoute += globalDemands[group - 1] # globalDemands tem indexes de 0 até nGroups - 1 
        if demandOfRoute > capacity :
            return False
    return True

# def calculateDistanceAfterChange(route, ...??):

def calculateDemands(routes):
    demandsRoutes = []
    routes = routes
    for route in routes:
        demandRoute = 0
        for group in route:
            demandRoute += globalDemands[group - 1] # globalDemands tem indexes de 0 até nGroups - 1 
        demandsRoutes.append(demandRoute)
    return demandsRoutes

########### FW
def novasDistancias(distances, route, nNodes, groups):
    nos = [0]
    novasDistancias = np.full((nNodes,nNodes), 100000000000) # copiamos matriz de distancias
    for i in range(len(route) - 1): # e mudamos valores para infinito quando nao sao do grupo
        group = route[i]
        next_group = route[i+1]
        nos_grupo = groups[next_group]
        nos += nos_grupo
        grupo1 = groups[group]
        grupo2 = groups[next_group]
        for node in grupo1:
            for i in range(nNodes):
                if i in grupo2:
                    novasDistancias[node][i] = distances[node][i]
                    #novasDistancias[i][node] = distances[node][i]
    return novasDistancias
        
def FloydWarshall(distances, route):
    newDistances = novasDistancias(distances, route, nNodes, groups)
    parent = []
    v = len(newDistances)
    # path reconstruction matrix
    p = np.zeros(newDistances.shape)
    for i in range(0,v):
        for j in range(0,v):
            p[i,j] = newDistances[i,j]

    # initialize to infinity
    for i in range (0, v):
        parent.append([])
        for j in range (0, v):
            parent[i].append(0)

    # initialize the path matrix
    for i in range (0,v):
        for j in range (0,v):
            if newDistances[i][j] == float("inf"):
                parent[i][j] = 0
            else:
                parent[i][j] = i

    changed = 0
    for k in range(0,v):
        for i in range(0,v):
            for j in range(0,v):
                if p[i,j] > p[i,k] + p[k,j]:
                    p[i,j] = p[i,k] + p[k,j]
                    parent[i][j] = parent[k][j]
                    changed +=1
    #print("\n\n\n")
    return p
                
#FloydWarshall(distances, [0,2,11,6,0])

# recursive function to obtain the path as a string
def obtainPath(i, j, parent):
    if newDistances[i][j] == float("inf"):
        return " no path to "
    if parent[i][j] == i:
        return " "
    else :
        return obtainPath(i, parent[i][j]) + str(parent[i][j]) + obtainPath(parent[i][j], j)
    
    # display shortest paths 

    '''for i in range (0,v):
        for j in range (0,v):
            if i == 0 and j == 0:
                print ("From :", i)
                print (" To :", j)
                print ("Path :" +  str(i) + obtainPath(i,j) + str(j))
                print ("Distance :", p[i][j])
                print("\n")
                
    #print(pd.DataFrame(p))'''


def processFile(filename):
    print('Start processing file :', filename)

    #filename = 'GVRP/A-n33-k6-C11-V2.gvrp'
    instance = filename[:-5]
    delimiter = ' '
    print(instance)

    # No final, vamos ter:
    # nodes = [node1, node2, ..., nodenNodes] = [[x_pos_node1, y_pos_node1, group_node_1], ....]
    # com group_node entre [1, nGroups]

    # e globalDemands = [demand_group_1, demand_group_2, ..., demand_group_nGroups]
    with open(filename,'r') as file:
        csv_reader = csv.reader(file,
                               delimiter = delimiter)
        line_count = 0
        node_count = 0
        group_count = 0
        nNodes, nGroups, nVehicles, capacity = 0,0,0,0
        for row in csv_reader:
            if line_count == 2 :  # DIMENSION : nNodes
                nNodes = int(row[2])
                nodes = np.zeros((nNodes,3), dtype=np.int)
            elif line_count == 3 : # VEHICLES : nVehicles
                nVehicles = int(row[2])
            elif line_count == 4 : # GVRP_SETS : nGroups
                nGroups = int(row[2])
                groups = [[0]] # Para guardar uma lista com os grupos
                globalDemands = np.zeros((nGroups), dtype=np.int)
            elif line_count == 5 :
                capacity = int(row[2]) # CAPACITY : capacity
            elif line_count >= 8 and line_count <= 8 + nNodes +  - 1: # Informacoes sobre as posicoes dos nos => num x_pos y_pos
                nodes[node_count] = [int(row[1]),int(row[2]), 0] # Ainda nao temos info sobre o grupo do nó, no depot vai ter grupo 0 
                node_count += 1
            elif line_count >= 8 + nNodes + 1 and line_count <= 8 + nNodes + nGroups: # Informacoes sobre os grupos => num_grupo elem elem ... elem -1
                nNodesInGroup = len(row)
                group = []
                for numNode in range(1,nNodesInGroup - 1 ): # Para evitar o "-1" que mostra quando pular linha
                    nodeOfGroup = int(row[numNode]) - 1
                    nodes[nodeOfGroup][2]  = group_count + 1
                    group.append(nodeOfGroup)
                group_count += 1
                groups.append(group)
                if group_count == nGroups:
                    group_count = 0  # Reseteando no final para usar na leitura seguinte
            elif line_count >= 8 + nNodes + nGroups + 2 and line_count <= 8 + nNodes + 2*(nGroups + 1) - 1:
                globalDemands[group_count] = int(row[1])
                group_count += 1
            line_count += 1

    #print(nNodes, nVehicles, nGroups, capacity)
    #print(nodes)
    #print(groups)
    #print(globalDemands)  
    distances = np.full((nNodes,nNodes), 100000000000)

    for node in range(0,nNodes):
        for second_node in range(node+1,nNodes):
            if nodes[node][2] != nodes[second_node][2]: # Nao queremos arestas entre os nós de um mesmo grupo
                x1, x2 = nodes[node][0], nodes[second_node][0]
                y1, y2 = nodes[node][1], nodes[second_node][1]
                distance2 = (x2 - x1)**2 + (y2 - y1)**2
                distances[node][second_node] = math.sqrt(distance2)
                distances[second_node][node] = math.sqrt(distance2)
                
    #print(pd.DataFrame(distances).head(5))

    print('Ending processing file :', instance)
    return instance, nNodes, nVehicles, nGroups, capacity, nodes, groups, globalDemands, distances

####### GVRP
def gvrp(filename):
    global instance, nNodes, nVehicles, nGroups, capacity, nodes, groups, globalDemands, distances
    instance, nNodes, nVehicles, nGroups, capacity, nodes, groups, globalDemands, distances = processFile(filename)
    ## Data structures
    set_seed(12345678)
    distances = distances
    iterations = 0
    grand_iterations = 0

    # Time setting
    inicio = timeit.default_timer()
    fim = timeit.default_timer() - 15
    time = fim - inicio
    #print(inicio,fim,time)
    print("\n")


    ## ILS
    ils_parameter = 1
    min_total_distance = 1000000
    routes = []
    demandsRoutes =  []
    #while(time < 20): # 2 seconds max
    ## Create initial solution
    # routes_initial = createInitialSolutionRandomly(nVehicles, nGroups)
    #routes_initial, demand_distribution = createInitialSolutionByBruteForce(nVehicles, nGroups, capacity, globalDemands)
    routes_initial, demand_distribution = createInitialSolutionBySweep(nVehicles, nGroups, capacity, globalDemands,nodes)
    print('initial routes : ', routes_initial)
    print('initial demand distribution : ', demand_distribution)
    distancesPerVehicle = calculateDistancesForRoutes(routes_initial, distances)
    print('initial distances : ', distancesPerVehicle)
    routes, distancesPerRoute = localSearch(routes_initial, distancesPerVehicle)
    total_distance = sum(distancesPerRoute)
    min_total_distance = total_distance
    demandsRoutes = calculateDemands(routes)
    print("at the end of the first local search:")
    print('routes : ', routes)
    print('demand distribution : ', demandsRoutes)
    distancesPerVehicle = calculateDistancesForRoutes(routes, distances)
    print('distances : ', distancesPerVehicle)
    print('min distance : ', min_total_distance)
    # Applying ils_parameter perturbations
    numberOfPerturbationsDone = 0
    #for i in range(ils_parameter): 
    fim = timeit.default_timer()
    time = fim - inicio
    print("\n")
    timeMax = 120
    while (time < timeMax):
        print("new iteration")
        perturbated_routes, distancesPerVehicleIteration, numberOfPerturbationsDone, time = perturbate2(routes,numberOfPerturbationsDone, distancesPerVehicle, time,timeMax)
        #distancesPerVehicleIteration = calculateDistancesForRoutes(perturbated_routes, distances)
        if time > timeMax:
            break
        new_routes, new_distances = localSearch(perturbated_routes, distancesPerVehicleIteration)
        new_distance = sum(new_distances)
        if new_distance < min_total_distance:
            min_total_distance = new_distance
            routes = new_routes
            distancesPerVehicle = new_distances
            demandsRoutes = calculateDemands(routes)
        iterations += 1
        # Checking if the solution of this iteration is better than the global solution
    #if min_total_distance_iteration < min_total_distance:
    #    min_total_distance = min_total_distance_iteration
    #    routes = routes_iteration
    #    demandsRoutes = calculateDemands(routes)
    #grand_iterations += 1 
        fim = timeit.default_timer()
        time = fim - inicio
        print("at the end of this iteration : ")
        print("routes : ", routes)
        print("demands : ", demandsRoutes)
        print("min distance : ", min_total_distance)
        print("\n")

    print("END : ")
    print('seconds : ', time)
    print('number of iterations : ', iterations)
    print('routes : ', routes)
    print('distancesPerVehicle : ',distancesPerVehicle)
    print('min_total_distance : ',min_total_distance)
    print('demandsRoutes : ',demandsRoutes)
    print('numberOfPerturbationsDone : ',numberOfPerturbationsDone)
    print("\n")
    print("\n")
    print("-------------------------------")
    print("\n")
    print("\n")
    writeToExcel(instance, nNodes, nVehicles, nGroups, capacity, min_total_distance, time)


    
#### WRITE TO EXCEL (results)
def writeToExcel(instance, nNodes, nVehicles, nGroups, capacity, min_total_distance, time):
    #rowDataCSV = "Instance|Nodes|Groups|Capacity|Vehicles|Solution|Time|Best Solution"
    rowDataCSV = instance + "|" + str(nNodes) + "|" + str(nGroups) + "|" + str(nVehicles) + "|" + str(capacity) + "|" + str(min_total_distance) + "|" + str(time)
    with open('GVRP/GVRP.csv','a') as fd:
        fd.write(rowDataCSV)
        fd.write("\n")
    
if __name__ == "__main__":
    gvrp('GVRP/A-n32-k5-C11-V2.gvrp') # Implement seed


Start processing file : GVRP/A-n32-k5-C11-V2.gvrp
A-n32-k5-C11-V2
Ending processing file : A-n32-k5-C11-V2


initial routes :  [[4, 6, 11, 2, 10, 1, 5], [8, 9, 7, 3]]
initial demand distribution :  [97, 42]
initial distances :  [330. 253.]
local search made x iterations with x =  1
at the end of the first local search:
routes :  [[4, 2, 11, 10, 6, 1, 5], [8, 7, 3, 9]]
demand distribution :  [97, 42]
distances :  [264. 204.]
min distance :  468.0
new iteration
found a solution when perturbating :  [4, 11, 10, 6, 1, 5] [8, 7, 3, 9, 2]
local search made x iterations with x =  1
at the end of this iteration : 
routes :  [[4, 11, 10, 6, 1, 5], [8, 7, 3, 9, 2]]
demands :  [83, 56]
min distance :  466.0


new iteration
found a solution when perturbating :  [8, 7, 3, 9] [4, 11, 10, 6, 1, 5, 2]
local search made x iterations with x =  1
at the end of this iteration : 
routes :  [[4, 11, 10, 6, 1, 5, 2], [8, 7, 3, 9]]
demands :  [97, 42]
min distance :  454.0


new iteration
found a solution whe